# Universo / Target

## Definición de target

Target = 1 cuando el alumno no curso ni rindió finales por 2 ciclos lectivos consecutivos y no haya finalizado la carrera

## Dataset a armar

ejemplo

| Alumno | Ciclo | Target |
| --- | --- | --- |
| 1 | 2010 | 0 |
| 1 | 2011 | 0 |
| 1 | 2012 | 1 |
| 1 | 2013 | 1 |
| 2 | 2012 | 0 |
| 2 | 2013 | 0 |
| 2 | 2014 | 0 |
| 2 | 2015 | 0 |

- ciclo minimo es el año de ingreso del alumno
- ciclo maximo es el año de egreso del alumno o del ultimo registro (cursadas o finales) + 2




In [1]:
import pandas as pd
import numpy as np

In [2]:
from ydata_profiling import ProfileReport

/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


## Datos de entrada

In [3]:
curdf = pd.read_pickle('../data/raw_cleaned/curdf_01.pkl')
print('cur shape: ',curdf.shape)
curdf_comp = pd.read_pickle('../data/raw_cleaned/curdf_complementario_01.pkl')
print('cur_comp shape: ',curdf_comp.shape)
datdf = pd.read_pickle('../data/raw_cleaned/datdf_01.pkl')
print('datdf shape: ',datdf.shape)
findf = pd.read_pickle('../data/raw_cleaned/findf_01.pkl')
print('fin shape: ',findf.shape)

cur shape:  (510290, 8)
cur_comp shape:  (79565, 8)
datdf shape:  (22157, 5)
fin shape:  (475362, 7)


## Reduccion de alumnos

- solo quedan los alumnos de las carreras:

|cod|carrera|alumnos|
|---|----------|----------|
| K | sistemas |     8702 |
|R|electronica|     3278|
|S|mecanica     |2822|
|V|quimica     |2475|
|O|civil     |2045|
|Q|eléctrica|      928|

In [4]:
datdf.columns

Index(['codigo_alumno', 'sexo', 'anio_nacimiento', 'anio_ingreso',
       'codigo_carrera_ingreso'],
      dtype='object')

In [5]:
datdf_aux = datdf[datdf.codigo_carrera_ingreso.isin(['K','R','S','V','O','Q'])].copy()
datdf_aux.shape


(20250, 5)

In [8]:
curdf['dataset'] = 'cursadas'

curdf_aux = curdf[curdf.codigo_alumno.isin(datdf_aux.codigo_alumno)].copy()
curdf_aux.shape

(465939, 9)

In [9]:
curdf_comp['dataset'] = 'cursadas_complementario'

curdf_comp_aux = curdf_comp[curdf_comp.codigo_alumno.isin(datdf_aux.codigo_alumno)].copy()
curdf_comp_aux.shape

(66900, 9)

In [11]:
findf['dataset'] = 'finales'

findf_aux = findf[findf.codigo_alumno.isin(datdf_aux.codigo_alumno)].copy()
findf_aux.shape

(431968, 8)

### condición

- tener en cuenta la materia de proyecto final

Proyecto Final:

	- civil (O): 950289

	- eléctrica (Q): 950594

	- electrónica (R): 950459

	- mecánica (S): 940842

	- química (V): 951196

	- sistemas (K): 082037

### datasets

| codigo_alumno | anio_ingreso | ultima_cursada | ultimo_final | egreso_anio |
| --- | --- | --- | --- | --- |

In [16]:
CODIGOS_MATERIAS_PF = ['950289','950594','950459','940842','951196','082037']
CODIGOS_CARRERAS = ['O','Q','R','S','V','K']

codigos = pd.DataFrame.from_dict({'codigo_materia':CODIGOS_MATERIAS_PF,'codigo_carrera':CODIGOS_CARRERAS})
codigos

,codigo_materia,codigo_carrera
0,950289,O
1,950594,Q
2,950459,R
3,940842,S
4,951196,V
5,082037,K


In [18]:
cur = pd.concat([curdf_aux,curdf_comp_aux],axis=0)
cur.head()

,codigo_alumno,curso,codigo_materia,departamento,turno,ciclo_lectivo,cant_cursada,cant_total_cursadas,dataset
4494,10035,K4153,950310,Unidad Docente Básica Legislación y Economía,Noche,2006.0,1.0,1,cursadas
4497,10035,K3051,952028,NaN,Noche,2007.0,1.0,1,cursadas
4492,10035,K3001,952030,NaN,Mañana,2006.0,1.0,1,cursadas
4498,10035,K3071,952031,NaN,Noche,2007.0,1.0,1,cursadas
4502,10035,K4051,952033,Departamento de Ing. en Sistemas de Información,Noche,2008.0,1.0,1,cursadas


In [19]:
VARS = ['codigo_alumno','ciclo_lectivo','codigo_materia','codigo_carrera']

In [20]:
cur_pf = pd.merge(cur,codigos,on=['codigo_materia','codigo_materia'],how='inner')[VARS]
cur_pf.head()

,codigo_alumno,ciclo_lectivo,codigo_materia,codigo_carrera
0,11123,2023.0,950459,R
1,34147,2007.0,950459,R
2,34147,2008.0,950459,R
3,34443,2007.0,950459,R
4,34470,2008.0,950459,R


In [21]:
print(cur_pf.codigo_alumno.shape)
print(cur_pf.codigo_alumno.nunique())

(4143,)
3551


In [45]:
cur_pf_aux = cur_pf.groupby(['codigo_alumno','codigo_carrera'])['ciclo_lectivo'].max().reset_index().copy()
cur_pf_aux.head()

,codigo_alumno,codigo_carrera,ciclo_lectivo
0,11123,R,2023.0
1,18230,Q,2015.0
2,2455,S,2000.0
3,30398,V,2010.0
4,33434,K,2016.0


In [46]:
print(cur_pf_aux.codigo_alumno.shape)
print(cur_pf_aux.codigo_alumno.nunique())

(3551,)
3551


#### conclusion

todos los alumnos que cursaron proyecto final lo hicieron en una sola carrera

In [24]:
findf_aux.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota_num',
       'nota_cual', 'nota_txt', 'cant_finales', 'dataset'],
      dtype='object')

In [26]:
VARS = ['codigo_alumno', 'anio_academico', 'codigo_materia', 'codigo_carrera']

In [27]:
fin_pf = pd.merge(findf_aux,codigos,on=['codigo_materia','codigo_materia'],how='inner')[VARS]

In [29]:
print(fin_pf.shape)
print(fin_pf.codigo_alumno.nunique())

(1539, 4)
1515


In [47]:
fin_pf_aux = fin_pf.groupby(['codigo_alumno','codigo_carrera'])['anio_academico'].max().reset_index().copy()
fin_pf_aux.head()

,codigo_alumno,codigo_carrera,anio_academico
0,18230,Q,2015.0
1,2455,S,2000.0
2,34147,R,2009.0
3,34437,R,2009.0
4,34443,R,2007.0


In [48]:
print(fin_pf_aux.shape)
print(fin_pf_aux.codigo_alumno.nunique())

(1515, 3)
1515


#### conclusion

los alumnos que rindieron final lo hicieron para una carrera sola

In [49]:
fin_pf.codigo_alumno.duplicated().value_counts()

False    1515
True       24
Name: codigo_alumno, dtype: int64

In [50]:
aux = fin_pf[fin_pf.codigo_alumno.isin(fin_pf[fin_pf.codigo_alumno.duplicated()]['codigo_alumno'])].copy()

pd.merge(aux, findf, on=['codigo_alumno','codigo_materia', 'anio_academico'], how='inner').drop_duplicates()

,codigo_alumno,anio_academico,codigo_materia,codigo_carrera,nota_num,nota_cual,nota_txt,cant_finales,dataset
0,53038,2018.0,950594,Q,9.0,NaN,Aprobada,1.0,finales
1,53038,2018.0,950594,Q,NaN,NaN,No Aprobada,2.0,finales
4,54194,2021.0,950594,Q,8.0,NaN,Aprobada,1.0,finales
5,54194,2021.0,950594,Q,NaN,NaN,No Aprobada,2.0,finales
8,41629,2017.0,950459,R,8.0,NaN,Aprobada,1.0,finales
9,41629,2017.0,950459,R,NaN,NaN,No Aprobada,2.0,finales
12,47623,2018.0,950459,R,10.0,NaN,Aprobada,1.0,finales
13,47623,2018.0,950459,R,NaN,NaN,No Aprobada,2.0,finales
16,49232,2018.0,950459,R,10.0,NaN,Aprobada,1.0,finales
17,49232,2018.0,950459,R,NaN,NaN,No Aprobada,2.0,finales


#### conclusion

los finales duplicados son ausentes

## Armado del dataset

| codigo_alumno | anio_ingreso | ultima_cursada | ultimo_final | egreso_anio |
| --- | --- | --- | --- | --- |

In [51]:
datdf_aux.columns

Index(['codigo_alumno', 'sexo', 'anio_nacimiento', 'anio_ingreso',
       'codigo_carrera_ingreso'],
      dtype='object')

In [62]:
print(datdf_aux.shape)
print(datdf_aux.codigo_alumno.nunique())

(20250, 5)
20250


In [53]:
# aux1 = datdf[['codigo_alumno','anio_ingreso']].copy()

In [54]:
cur_pf_aux.columns

Index(['codigo_alumno', 'codigo_carrera', 'ciclo_lectivo'], dtype='object')

In [56]:
aux2 = cur_pf_aux[['codigo_alumno','ciclo_lectivo']].copy()

In [57]:
fin_pf_aux.columns

Index(['codigo_alumno', 'codigo_carrera', 'anio_academico'], dtype='object')

In [58]:
aux3 = fin_pf_aux[['codigo_alumno','anio_academico']].copy()

In [64]:
# juntamos los 3 datasets

df_pf = pd.merge(aux2, aux3, on='codigo_alumno', how = 'left' )

df_pf = pd.merge(df_pf, datdf_aux, on = 'codigo_alumno', how = 'inner' )

df_pf.head()

,codigo_alumno,ciclo_lectivo,anio_academico,sexo,anio_nacimiento,anio_ingreso,codigo_carrera_ingreso
0,11123,2023.0,NaN,M,1967,1993,R
1,18230,2015.0,2015.0,M,1961,1984,Q
2,2455,2000.0,2000.0,M,1973,1992,S
3,30398,2010.0,NaN,M,1986,2005,S
4,33434,2016.0,NaN,M,1980,1999,K


In [65]:
df_pf.codigo_alumno.nunique()

3551

## Nexts Steps

- tengo el dataset de ultimas cursadas y ultimos finales dados
---
- [ ] TODO: TARGET. falta armar el dataset con el target. 
    - tener en cuenta el año de la cursada + 2 para q sea desertor
    - si rindio el final, egreso y no deberia contemplarse como desertor a pesar de haberse recibido antes

---
- [ ] TODO: DATASET
    - 1. a nivel alumno
    - 2. a nivel ciclo lectivo para hacerlo temporal